In [1]:
import argparse
import os, sys
import logging
import joblib 
import numpy as np
import pandas as pd 
import joblib
import uproot 
import awkward as ak
from pathlib import Path


In [2]:
pkls_path = '/global/cfs/projectdirs/atlas/hrzhao/HEP_Repo/QG_Calibration/tmp/pythiaA_pkl'
pkls_path = Path(pkls_path)

In [3]:
pkl_files = sorted(pkls_path.glob("*.pkl"))
pkl_path = pkl_files[10]
pkl_file = joblib.load(pkl_files[2])

In [ ]:
# 300 -> 300 with scores 
# 300 -> reweighting function  

# 300 -> 300 


In [ ]:
pkl_file

In [6]:
gbdt_path = '/global/cfs/projectdirs/atlas/hrzhao/HEP_Repo/QG_Calibration/BDT_EB4/LightGBM/optuna_tuning/small_dataset/lightgbm_gbdt.pkl'
gbdt_path = Path(gbdt_path)
gbdt = joblib.load(gbdt_path)

In [ ]:
training_vars = ['jet_pt', 'jet_nTracks', 'jet_trackWidth', 'jet_trackC1']


In [ ]:
pred = gbdt.predict(pkl_file[training_vars], raw_score= True)


In [ ]:
from matplotlib import pyplot as plt
plt.hist(pred)

In [4]:
from pkl2predpkl import * 

In [7]:
pkl2predpkl(pkl_path=pkl_path, gbdt_path=gbdt_path)

,run,event,jet_pt,jet_eta,jet_nTracks,jet_trackWidth,jet_trackC1,jet_trackBDT,jet_PartonTruthLabelID,event_weight,is_forward,is_leading,pt_idx,target,GBDT_newScore
0,364707.0,13042655.0,1467.928467,-0.419321,17.0,0.029595,0.087022,-0.235556,1.0,0.003446,0.0,1.0,4,0,-2.583729
1,364707.0,13042655.0,1422.241577,0.678834,41.0,0.049013,0.227557,0.299710,1.0,0.003446,1.0,0.0,4,0,1.204736
2,364707.0,13811564.0,1951.648682,0.053474,9.0,0.009728,0.138308,-0.516583,2.0,0.001457,0.0,1.0,5,0,-4.267301
3,364707.0,13811564.0,1809.555420,0.515598,36.0,0.060653,0.246813,0.141547,21.0,0.001457,1.0,0.0,5,1,0.013235
4,364707.0,13814657.0,1867.564453,-0.016487,17.0,0.007103,0.043331,-0.260059,2.0,0.001730,0.0,1.0,5,0,-2.738410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233055,364707.0,13874522.0,1762.776001,0.593770,39.0,0.029674,0.216912,0.321189,2.0,0.001287,1.0,0.0,5,0,1.108991
233056,364707.0,13875091.0,1855.234131,-1.010485,18.0,0.019418,0.182468,-0.092021,2.0,0.002059,1.0,1.0,5,0,-1.801626
233057,364707.0,13875091.0,1734.411987,-0.199658,36.0,0.021826,0.086786,0.183787,21.0,0.002059,0.0,0.0,5,1,0.232641
233058,364707.0,13864087.0,1854.308594,-0.937280,16.0,0.010622,0.131295,-0.264008,2.0,0.000582,1.0,1.0,5,0,-2.967816


# test parallel

In [9]:
from concurrent.futures import ProcessPoolExecutor
import functools 

In [10]:

pkl2predpkl_mod = functools.partial(pkl2predpkl,gbdt_path=gbdt_path, if_save=False)
        
with ProcessPoolExecutor(max_workers=8) as executor:
        pred_pkls_list = list(executor.map(pkl2predpkl_mod, pkl_files[0:10])) 

In [ ]:
pred_pkls_list